In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv('TCS2018.csv')
df.head()

,Symbol,Series,Date,Prev Close,Open Price,High Price,Low Price,Last Price,Close Price,Average Price,Total Traded Quantity,Turnover,No. of Trades,Deliverable Qty,% Dly Qt to Traded Qty
0,TCS,EQ,01-Jan-2018,2701.20,2682.3,2694.8,2635.0,2650.0,2645.60,2654.94,675880,1.794418e+09,19464,330691,48.93
1,TCS,EQ,02-Jan-2018,2645.60,2660.0,2669.6,2620.2,2631.0,2631.20,2641.06,960145,2.535800e+09,46813,555820,57.89
2,TCS,EQ,03-Jan-2018,2631.20,2632.0,2669.0,2631.2,2639.0,2638.65,2649.21,628560,1.665186e+09,32104,361072,57.44
3,TCS,EQ,04-Jan-2018,2638.65,2650.0,2662.0,2640.0,2650.0,2657.10,2649.94,456541,1.209807e+09,32647,264561,57.95
4,TCS,EQ,05-Jan-2018,2657.10,2650.0,2699.5,2650.0,2695.0,2689.20,2684.95,576853,1.548821e+09,30836,318083,55.14


In [3]:
remove_columns = ['Symbol', 'Series', 'Last Price', 'Average Price', 'Total Traded Quantity','Turnover',
                 'No. of Trades','Deliverable Qty','% Dly Qt to Traded Qty']
clean_df = df.drop(remove_columns, axis=1)
clean_df.head()

,Date,Prev Close,Open Price,High Price,Low Price,Close Price
0,01-Jan-2018,2701.20,2682.3,2694.8,2635.0,2645.60
1,02-Jan-2018,2645.60,2660.0,2669.6,2620.2,2631.20
2,03-Jan-2018,2631.20,2632.0,2669.0,2631.2,2638.65
3,04-Jan-2018,2638.65,2650.0,2662.0,2640.0,2657.10
4,05-Jan-2018,2657.10,2650.0,2699.5,2650.0,2689.20


In [4]:
clean_df['Avg_high_5'] = clean_df['High Price'].rolling(5).mean()
clean_df['Avg_low_5'] = clean_df['Low Price'].rolling(5).mean()
clean_df['MV_5'] = clean_df['Close Price'].rolling(5).mean()
clean_df[:10]

,Date,Prev Close,Open Price,High Price,Low Price,Close Price,Avg_high_5,Avg_low_5,MV_5
0,01-Jan-2018,2701.20,2682.3,2694.80,2635.00,2645.60,NaN,NaN,NaN
1,02-Jan-2018,2645.60,2660.0,2669.60,2620.20,2631.20,NaN,NaN,NaN
2,03-Jan-2018,2631.20,2632.0,2669.00,2631.20,2638.65,NaN,NaN,NaN
3,04-Jan-2018,2638.65,2650.0,2662.00,2640.00,2657.10,NaN,NaN,NaN
4,05-Jan-2018,2657.10,2650.0,2699.50,2650.00,2689.20,2678.98,2635.28,2652.35
5,08-Jan-2018,2689.20,2700.0,2726.85,2681.85,2714.40,2685.39,2644.65,2666.11
6,09-Jan-2018,2714.40,2715.0,2721.00,2676.55,2709.00,2695.67,2655.92,2681.67
7,10-Jan-2018,2709.00,2711.0,2816.00,2694.65,2806.60,2725.07,2668.61,2715.26
8,11-Jan-2018,2806.60,2810.0,2821.90,2782.20,2790.50,2757.05,2697.05,2741.94
9,12-Jan-2018,2790.50,2805.0,2805.00,2740.55,2776.35,2778.15,2715.16,2759.37


In [5]:
# plt.figure(figsize=(15,10))
# plt.grid(True)
# plt.plot(clean_df['Avg_high_5'], label='AVG_HIGH_5')
# plt.plot(clean_df['Avg_low_5'], label='AVG_LOW_5')
# plt.plot(clean_df['MV_5'], label='MV_5')
# plt.legend(loc=2)

In [12]:
row_count = clean_df.shape[0]
trades_columns = ['Date Entered', 'Price Entered', 'Date Exited', 'Price Exited', 'Long/Short', 'Points Captured']
trade_id = 0

current_trade = 0 # 0 for no position, 1 for long, -1 for short

trades_df = pd.DataFrame(columns = trades_columns)

for i in range(row_count):
    
    if current_trade==0 and clean_df['Close Price'][i] > clean_df['Avg_high_5'][i]:
        # Trigger Long Position
        current_trade = 1
        # Insert row with index=trade_id and data type of missing values as required
        trades_df.loc[trade_id] = [clean_df['Date'][i], clean_df['Close Price'][i], "Not Closed", -1, 'Long', 0]
        
    elif current_trade==0 and clean_df['Close Price'][i] < clean_df['Avg_low_5'][i]:
        # Trigger Short Position
        current_trade = -1
        # Insert row with index=trade_id and data type of missing values as required
        trades_df.loc[trade_id] = [clean_df['Date'][i], clean_df['Close Price'][i], "Not Closed", -1, 'Short', 0]
        
    elif current_trade==1 and clean_df['Close Price'][i] < clean_df['Avg_high_5'][i]:
        # Close Long Position
        current_trade = 0
        trades_df.at[trade_id, 'Date Exited'] = clean_df['Date'][i]
        trades_df.at[trade_id, 'Price Exited'] = clean_df['Close Price'][i]
        trades_df.at[trade_id, 'Points Captured'] = trades_df['Price Exited'][trade_id] - trades_df['Price Entered'][trade_id]
        trade_id += 1
        
    elif current_trade==-1 and clean_df['Close Price'][i] > clean_df['Avg_low_5'][i]:
        # Close Short Position
        current_trade = 0
        trades_df.at[trade_id, 'Date Exited'] = clean_df['Date'][i]
        trades_df.at[trade_id, 'Price Exited'] = clean_df['Close Price'][i]
        trades_df.at[trade_id, 'Points Captured'] = trades_df['Price Entered'][trade_id] - trades_df['Price Exited'][trade_id]
        trade_id += 1
        
trades_df

,Date Entered,Price Entered,Date Exited,Price Exited,Long/Short,Points Captured
0,05-Jan-2018,2689.20,12-Jan-2018,2776.35,Long,87.15
1,16-Jan-2018,2850.85,25-Jan-2018,3121.05,Long,270.2
2,29-Jan-2018,3198.85,30-Jan-2018,3152.85,Long,-46
3,06-Feb-2018,2997.25,12-Feb-2018,2967.3,Short,29.95
4,14-Feb-2018,2911.80,15-Feb-2018,2925.85,Short,-14.05
5,21-Feb-2018,3042.30,26-Feb-2018,3031.7,Long,-10.6
6,05-Mar-2018,3105.85,06-Mar-2018,3042.8,Long,-63.05
7,08-Mar-2018,3003.95,09-Mar-2018,3034.1,Short,-30.15
8,13-Mar-2018,2886.80,20-Mar-2018,2864.85,Short,21.95
9,23-Mar-2018,2818.15,26-Mar-2018,2817,Short,1.15


In [14]:
trades_df['Points Captured'].sum()

304.150000000001